In [20]:
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_community.llms import LlamaCpp

from dotenv import load_dotenv, find_dotenv
import os
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.question_answering import load_qa_chain
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain_community.embeddings import HuggingFaceInstructEmbeddings
from langchain_community.embeddings import HuggingFaceEmbeddings

In [17]:
load_dotenv(find_dotenv())
HUGGINGFACE_API_KEY = os.environ.get("SECRET_KEY")

In [2]:
loader = PyPDFLoader("/media/sf_VMShare/C13_Networking.pdf")

#Load the document by calling loader.load()
pages = loader.load()

print(len(pages))
print(pages[0].page_content[0:500])

print(pages[0].metadata)

100
620Fundamentals of
Networking and
Network Protocols
Imagine that your computer has no connectivity to the Internet. Would we consider such
a computer fully functional? Probably not. Although we take the Internet and network
connectivity for granted, it is a revelation to learn how we got to this point in the first
place. We will do such a historical review at the end of this chapter. First, we will under-
stand the basic elements of networking that allow computers to talk to one another, be
they
{'source': '/media/sf_VMShare/C13_Networking.pdf', 'page': 0}


In [3]:
rsplitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=10)
splits = rsplitter.split_documents(pages)
print(len(splits))
print(len(pages))

288
100


In [19]:
# Define the path to the pre-trained model you want to use
modelPath = "sentence-transformers/all-MiniLM-l6-v2"

# Create a dictionary with model configuration options, specifying to use the CPU for computations
model_kwargs = {'device':'cpu'}

# Create a dictionary with encoding options, specifically setting 'normalize_embeddings' to False
encode_kwargs = {'normalize_embeddings': False}

# Initialize an instance of HuggingFaceEmbeddings with the specified parameters
embeddings = HuggingFaceEmbeddings(
    model_name=modelPath,     # Provide the pre-trained model's path
    model_kwargs=model_kwargs, # Pass the model configuration options
    encode_kwargs=encode_kwargs # Pass the encoding options
)

modules.json: 100%|██████████| 349/349 [00:00<00:00, 1.37MB/s]


TypeError: INSTRUCTOR._load_sbert_model() got an unexpected keyword argument 'token'

In [ ]:
# Create the vector store
persist_directory = 'docs/chroma/'
vectorDB = Chroma.from_documents(documents=splits,
                                     embedding=embeddings,
                                     persist_directory=persist_directory)
print(vectorDB._collection.count())

In [ ]:
# Callbacks support token-wise streaming
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

# Make sure the model path is correct for your system!
llm = LlamaCpp(
    model_path="/media/sf_VMShare/zephyr-7b-beta.Q2_K.gguf",
    temperature=0.75,
    max_tokens=2000,
    top_p=1,
    callback_manager=callback_manager,
    verbose=True,  # Verbose is required to pass to the callback manager
)

In [ ]:
# Build prompt
template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible. Always say "thanks for asking!" at the end of the answer.
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)# Run chain
qaChain = RetrievalQA.from_chain_type(
        llm,
        retriever=vectorDB.as_retriever(),
        return_source_documents=True,
        chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
    )

In [ ]:
def AskMe(question):
  result = qaChain({"query": question})
  return result["result"]

In [ ]:
# Pass question to the qa_chain
question = "What are major topics for this document?"
print(AskMe(question))